## In-class activity 1: LLM Environment Setup

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
   !pip install unsloth
else:
   # Do this only in Colab notebooks! Otherwise use pip install unsloth
   !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
   !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
   !pip install --no-deps unsloth

In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())  # → True
print("GPU count:", torch.cuda.device_count())       # → 1 이상
print("GPU name:", torch.cuda.get_device_name(0))     # → RTX 408

ImportError: Failed to load PyTorch C extensions:
    It appears that PyTorch has loaded the `torch/_C` folder
    of the PyTorch repository rather than the C extensions which
    are expected in the `torch._C` namespace. This can occur when
    using the `install` workflow. e.g.
        $ python setup.py install && python -c "import torch"

    This error can generally be solved using the `develop` workflow
        $ python setup.py develop && python -c "import torch"  # This should succeed
    or by running Python from a different directory.

In [2]:
from unsloth import FastLanguageModel

fourbit_models = [
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    device_map = 8192,
    load_in_4bit = True,
)

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((

In [ ]:
messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenizer =True,
    add_generation_prompt = True,
    return_tensors = "pt"
    ).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_texts)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Describe a tall tower in the capital of France.assistant

I do not have the capability to search or verify the location, but I can describe a hypothetical, tall, tower that is located in the capital of France.

Imagine a sleek, modern tower that stands majestically in the heart of Paris, the capital of France. This towering structure stands at approximately 75 to 100 meters tall, and its height is impressive to say the least.

As you approach the tower, its slender design blends seamlessly into the modern architecture of Paris. The tower's foundation is carefully anchored, allowing the structure to expand outward and downward to ensure stability.

From the outside, the tower's facade features a large glass viewing floor on the 20th floor. This glass floor offers panoramic views of the City of Light, allowing those inside to witness the beauty and energy of Paris.

Below the tower, a sleek and efficient networ

In [ ]:
from transformers import TextStreamer
messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenizer =True,
    add_generation_prompt = True,
    return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    input_ids = inputs,
    streamer =text_streamer,
    max_new_tokens = 1024,
    use_cache = True,
    temperature = 1.5,
    min_p = 0.1,
)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Describe a tall tower in the capital of France.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I believe you are referring to the Eiffel Tower in the city of Paris, the capital of France.

The Eiffel Tower stands approximately 1,063 feet (324 meters) tall, with a total height including its iron structure of 1,050 feet. It's a striking example of engineering innovation and a symbol of the city and France. The tower is currently occupied by tourists, vendors, and workers due to its high maintenance costs.

Built between 1887 and 1889 as an entrance arch for the Exposition Universelle (World's Fair), it was intended to last only a few years. Instead, the tower became a permanent destination for millions of visitors, and it is now one of the most iconic landmarks in the world.

The Eiffel Tower was the larg

In [ ]:
messages = [
    {"role": "user",
     "content": """Rewrite the following sentence in active voice:
     "The cake was eaten by the chile."
     """}]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenizer =True,
    add_generation_prompt = True,
    return_tensors = "pt"
    ).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_texts)

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Rewrite the following sentence in active voice:
     "The cake was eaten by the chile."
     assistant

Here's the rewritten sentence in active voice:

"Someone (specifically, a person) was eaten by the cake."

Or a more idiomatic version:

"The cake was eaten, leaving it destroyed."

This revised sentence clearly states who or what is performing the action of eating the cake.


In [ ]:
messages = [
    {
        "role": "user",
        "content": """Rewrite the following sentence in active voice.

Input: The ball was thrown by sarah.
Output: Sarah threw the ball.

Input: The ball was thrown by sarah.
Output: Sarah threw the ball.

Input: The cake was eaten by the child.
Output:
     """}]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenizer =True,
    add_generation_prompt = True,
    return_tensors = "pt"
    ).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_texts)

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Rewrite the following sentence in active voice.

Input: The ball was thrown by sarah.
Output: Sarah threw the ball.

Input: The ball was thrown by sarah.
Output: Sarah threw the ball.

Input: The cake was eaten by the child.
Output:
     assistant

Rewriting the active voice is not required because the subject of the sentence does not need to be changed to active voice.

However, here are two active voice sentences to demonstrate:

- The sun rose early in the morning, rising above the horizon. 
- Sarah bought a cake for the birthday party. 

These two sentences will give you an active voice.


In [ ]:
messages = [
    {
        "role": "user",
        "content": """Summarize the following paragraph in one sentence:
"Artificial intelligence (AI) is a multidisciplinary field within compuiter science that is focus
     """}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenizer =True,
    add_generation_prompt = True,
    return_tensors = "pt"
    ).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_texts)

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Summarize the following paragraph in one sentence:
"Artificial intelligence (AI) is a multidisciplinary field within compuiter science that is focus
     assistant

The field of artificial intelligence (AI) is a multidisciplinary area of computer science that involves creating intelligent machines and systems, such as computers and robots.


In [ ]:
messages = [
    {
        "role": "user",
        "content": """You are a helpful and professional academic writing assistant.
Rewrite the following sentence in a more formal and academic tone:
"This study kinda shows that people tend to like working from home way more."
     """}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenizer =True,
    add_generation_prompt = True,
    return_tensors = "pt"
    ).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_texts)

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

You are a helpful and professional academic writing assistant.
Rewrite the following sentence in a more formal and academic tone:
"This study kinda shows that people tend to like working from home way more."
     assistant

This observational study appears to suggest a preference among respondents for working from home, suggesting an affirmative inclination towards remote work.

Alternatively, here are a few other options:

* This study provides evidence for the growing trend of working remotely, implying a positive sentiment towards remote work.
* This quantitative analysis of employee preferences indicates that a majority of individuals opt for working from home, thereby underscoring a trend towards remote work.
* The results of this study reveal that there is a significant proportion of respondents who prefer working from home, indicating a preference among individuals for remote work arrangements.

Please

In [ ]:
messages = [
    {
        "role": "user",
        "content": """
Prompt: Q: If a box has 4 apples and 6 oranges, and you pick one at
          random, what is the probability it’s an apple?
          A: Let's think step by step.
          There are 4 apples + 6 oranges = 10 total.
          Probability of apple = 4 / 10 = 0.4
          Answer: 0.4

          Q: If there are 3 red balls and 2 blue balls in a box, and  take one out at random, what is the probability it’s red?
          A: Let's think step by step.
     """}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenizer =True,
    add_generation_prompt = True,
    return_tensors = "pt"
    ).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_texts)

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user


Prompt: Q: If a box has 4 apples and 6 oranges, and you pick one at 
          random, what is the probability it’s an apple? 
          A: Let's think step by step. 
          There are 4 apples + 6 oranges = 10 total. 
          Probability of apple = 4 / 10 = 0.4 
          Answer: 0.4 

          Q: If there are 3 red balls and 2 blue balls in a box, and  take one out at random, what is the probability it’s red? 
          A: Let's think step by step.
     assistant

I can help you apply the concept of probability.


## Step 1:  Calculate the number of favorable outcomes
In this scenario, we have 3 red balls and 2 blue balls. To find the number of favorable outcomes, we'll assume a single ball has been drawn at random from the box.

## Step 2:  Identify the total number of balls in the box
After picking one ball at random from the box, we can see that it's actually not about finding the total number of b

In [ ]:
messages = [
    {
        "role": "user",
        "content": """
• Prompt: If there are 3 cars and each car has 4 wheels, how many
        wheels are there in total? Let's think step by step.
     """}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenizer =True,
    add_generation_prompt = True,
    return_tensors = "pt"
    ).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_texts)

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user


• Prompt: If there are 3 cars and each car has 4 wheels, how many 
        wheels are there in total? Let's think step by step.
     assistant

To find the total number of wheels in the 3 cars, let's break it down step by step.

**Step 1: Find the total number of wheels in one car.**
Each car has 4 wheels, so we multiply the number of wheels by 1 to get the number of wheels in one car: 4 wheels * 1 car = 4 wheels.

**Step 2: Find the total number of wheels in 3 cars.**
Since there are 3 cars, we multiply the number of wheels in one car by 3: 4 wheels * 3 cars = 12 wheels.

Therefore, there are 12 wheels in total.
